# Unwired Data Processing

Radex2000 database query from 2010 in the 3.4 GHz band.

## Calculating Total Power into Antenna (TPA)

TPA = Total Power into Antenna

EIRP/5

$G_a$ = Antenna Gain

$TPA_{dBW} = EIRP_{dBW} - G_{a(dBi)}$

$TPA_{dBW} = EIRP_{dBm} - 30 - G_{a(dBi)}$

Spectral Density $ = EIRP_{dBm} + 10log_{10}(\frac{1}{BW})$

### Unit Conversion Formulas

$P_{dbW} = 10 \times log_{10}(P_W)$

$P_W = 10^{\frac{P_{dBW}}{10}}$

$P_{dBm} = P_{dBW} + 30$

In [47]:
import pandas as pd
import numpy as np

unwired = pd.read_excel('Data/20103_4akal.xlsx')

# Remove search irrelevant data
unwired = unwired[[
    'Site_Mode',
    'EIRP',
    'Antenna_Ht',
    'Site_Name',
    'Location',
    'State',
    'Antenna_Type',
    'Antenna_Gain',
    'Lic_BW',
    'Licence_Sub_Type',
    'Freq_Upper',
    'Freq_Lower',
    'Antenna_Manuf',
    'Latitude',
    'Longitude'
    ]]

# Filter out Rx entries
unwired = unwired[unwired['Site_Mode'] == 'T']

# Filter out AIRSPAN antenna (back haul antennas)
unwired = unwired[unwired['Antenna_Manuf'] == 'NAVINI NETWORKS']

# Hz to MHz
unwired['Lic_BW'] = (unwired['Lic_BW'] / 1000000).round(3)
unwired['Freq_Upper'] = (unwired['Freq_Upper'] / 1000000).round(7)
unwired['Freq_Lower'] = (unwired['Freq_Lower'] / 1000000).round(7)
unwired.rename(columns={'Lic_BW':'BW (MHz)'}, inplace=True)

# Convert EIRP/30kHz to EIRP
unwired.rename(columns={'EIRP' : 'EIRP/30kHz_dBm'}, inplace=True)
unwired['EIRP_dBm'] = unwired['EIRP/30kHz_dBm'] - 10 * np.log10(0.03 / unwired['BW (MHz)'])
unwired['EIRP'] = 10 * np.power(10, (unwired['EIRP_dBm'] - 30))

# Convert W to dBm
# unwired['EIRP_dBm'] = 0
# unwired = unwired.astype({'EIRP' : 'float','EIRP_dBm' : 'float'})
# unwired = unwired.replace({0:np.nan})
# unwired['EIRP_dBm'] = 10 * np.log10(1000 * unwired['EIRP'])

# Convert dBm to dBm/MHz
unwired['EIRP_dBm/MHz'] = unwired['EIRP_dBm'] + 10 * np.log10(1 / unwired['BW (MHz)'])

# Total Power into Antenna
unwired['TPA_dBW'] = (unwired['EIRP_dBm'] - 30) - unwired['Antenna_Gain']
unwired['TPA_W'] = np.power(10, (unwired['TPA_dBW'] / 10))
unwired['TPA_dBm/MHz'] = (unwired['TPA_dBW'] + 30) + 10 * np.log10(1 / unwired['BW (MHz)'])
unwired['MHz_dBm'] = 10 * np.log10(1 / unwired['BW (MHz)'])

# Save to xlsx
unwired.reset_index(drop=True, inplace=True)
unwired.to_excel('Data/unwired.xlsx', index=False)
unwired

,Site_Mode,EIRP/30kHz_dBm,Antenna_Ht,Site_Name,Location,State,Antenna_Type,Antenna_Gain,BW (MHz),Licence_Sub_Type,...,Antenna_Manuf,Latitude,Longitude,EIRP_dBm,EIRP,EIRP_dBm/MHz,TPA_dBW,TPA_W,TPA_dBm/MHz,MHz_dBm
0,T,22.8,151,367 Collins Street Optus Centre MELBOURNE,MELBOURNE,VIC,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-37.818810,144.961039,45.018487,1.043488e+16,38.028787,-0.981513,0.797717,22.028787,-6.9897
1,T,22.8,151,367 Collins Street Optus Centre MELBOURNE,MELBOURNE,VIC,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-37.818810,144.961039,45.018487,1.043488e+16,38.028787,-0.981513,0.797717,22.028787,-6.9897
2,T,22.8,151,367 Collins Street Optus Centre MELBOURNE,MELBOURNE,VIC,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-37.818810,144.961039,45.018487,1.043488e+16,38.028787,-0.981513,0.797717,22.028787,-6.9897
3,T,22.8,151,367 Collins Street Optus Centre MELBOURNE,MELBOURNE,VIC,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-37.818810,144.961039,45.018487,1.043488e+16,38.028787,-0.981513,0.797717,22.028787,-6.9897
4,T,22.8,151,367 Collins Street Optus Centre MELBOURNE,MELBOURNE,VIC,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-37.818810,144.961039,45.018487,1.043488e+16,38.028787,-0.981513,0.797717,22.028787,-6.9897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,T,21.8,46,Northcott - East Wing Belvoir Street STRAWBE...,STRAWBERRY HILLS,NSW,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-33.890308,151.209834,44.018487,1.043488e+15,37.028787,-1.981513,0.633649,21.028787,-6.9897
359,T,21.8,27,Rooftop 49 Bennett Street BONDI,BONDI,NSW,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-33.898087,151.258669,44.018487,1.043488e+15,37.028787,-1.981513,0.633649,21.028787,-6.9897
360,T,21.8,27,Rooftop 49 Bennett Street BONDI,BONDI,NSW,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-33.898087,151.258669,44.018487,1.043488e+15,37.028787,-1.981513,0.633649,21.028787,-6.9897
361,T,21.8,33,Optus Site Corner of Phillip St & Copeland St ...,KINGSWOOD,NSW,Multi Element Panel (1 sector),16.0,5.0,3.4 GHz Upper Band B,...,NAVINI NETWORKS,-33.756141,150.715801,44.018487,1.043488e+15,37.028787,-1.981513,0.633649,21.028787,-6.9897


In [27]:
# Bandwidth Table
BWstats = pd.DataFrame(unwired['BW (MHz)'].value_counts(sort=False))
BWstats['Percentage (%)'] = (BWstats['BW (MHz)'] /  363) * 100
BWstats.to_csv('Data/BWstats.csv', index=True)
BWstats


,BW (MHz),Percentage (%)
5.0,336,92.561983
2.0,27,7.438017


In [28]:
# TPA table
tpa = pd.DataFrame(unwired['TPA_W'].value_counts( sort=False))
tpa['Percentage (%)'] = (tpa['TPA_W'] /  363) * 100
tpa.to_csv('Data/TPAtable.csv', index=True)
tpa


,TPA_W,Percentage (%)
0.797717,75,20.661157
0.050333,32,8.815427
0.317577,35,9.641873
0.503325,88,24.242424
0.633649,46,12.672176
0.126430,32,8.815427
0.079772,1,0.275482
0.012643,12,3.305785
0.252260,1,0.275482
0.004186,14,3.856749


In [29]:
# TPA Distribution (Watts)
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 1]
TPAdist = pd.DataFrame(unwired['TPA_W'].value_counts(bins=bins, sort=False))
TPAdist['Percentage (%)'] = (TPAdist['TPA_W'] /  363) * 100
TPAdist.rename(columns={'TPA_W' : 'Total Power into Antenna (W)'}, inplace=True)
TPAdist.to_csv('Data/TPAdist_W.csv', index=True)
TPAdist

,Total Power into Antenna (W),Percentage (%)
"(-0.001, 0.1]",59,16.253444
"(0.1, 0.2]",32,8.815427
"(0.2, 0.3]",1,0.275482
"(0.3, 0.4]",62,17.079890
"(0.4, 0.5]",0,0.000000
"(0.5, 0.6]",88,24.242424
"(0.6, 1.0]",121,33.333333


In [37]:
# TPA Distribution (dBm/MHz)
bins = [-1,0, 5, 10, 15, 20, 25]
TPAdist = pd.DataFrame(unwired['TPA_dBm/MHz'].value_counts(bins=bins, sort=False))
TPAdist['Percentage (%)'] = (TPAdist['TPA_dBm/MHz'] /  363) * 100
TPAdist.rename(columns={'TPA_W' : 'Total Power into Antenna (dBm/MHz)'}, inplace=True)
TPAdist.to_csv('Data/TPAdist_dBmMHz.csv', index=True)
TPAdist

,TPA_dBm/MHz,Percentage (%)
"(-1.001, 0.0]",14,3.856749
"(0.0, 5.0]",12,3.305785
"(5.0, 10.0]",0,0.000000
"(10.0, 15.0]",65,17.906336
"(15.0, 20.0]",36,9.917355
"(20.0, 25.0]",236,65.013774


In [31]:
# Antenna Gains
bins = [0, 15, 17.5, 20, 22.5, 25, 100]
AntennaG = pd.DataFrame(unwired['Antenna_Gain'].value_counts( sort=False))
AntennaG['Percentage (%)'] = (AntennaG['Antenna_Gain'] /  363) * 100
AntennaG.rename(columns={'Antenna_Gain' : 'Antenna Gain (dBi)'}, inplace=True)
AntennaG.to_csv('Data/AntennaGain.csv', index=True)
AntennaG

,Antenna Gain (dBi),Percentage (%)
16.0,363,100.0


In [41]:
# EIRP dBm
bins = [0,20, 25, 30, 35, 40, 45,100]
EIRPs_dBm = pd.DataFrame(unwired['EIRP_dBm'].value_counts(bins=bins, sort=False))
EIRPs_dBm['Percentage (%)'] = (EIRPs_dBm['EIRP_dBm'] /  363) * 100
EIRPs_dBm.rename(columns={'EIRP_dBm' : 'EIRP (dBm)'}, inplace=True)
EIRPs_dBm.to_csv('Data/EIRPs_dBm.csv', index=True)
EIRPs_dBm

,EIRP (dBm),Percentage (%)
"(-0.001, 20.0]",0,0.000000
"(20.0, 25.0]",14,3.856749
"(25.0, 30.0]",12,3.305785
"(30.0, 35.0]",32,8.815427
"(35.0, 40.0]",33,9.090909
"(40.0, 45.0]",197,54.269972
"(45.0, 100.0]",75,20.661157


In [45]:
# EIRP dBm/MHz
bins = [0,10, 20, 30, 35, 40,]
EIRPs_dBm = pd.DataFrame(unwired['EIRP_dBm/MHz'].value_counts(bins=bins, sort=False))
EIRPs_dBm['Percentage (%)'] = (EIRPs_dBm['EIRP_dBm/MHz'] /  363) * 100
EIRPs_dBm.rename(columns={'EIRP_dBm/MHz' : 'EIRP (dBm/MHz)'}, inplace=True)
EIRPs_dBm.to_csv('Data/EIRPs_dBmHz.csv', index=True)
EIRPs_dBm

,EIRP (dBm/MHz),Percentage (%)
"(-0.001, 10.0]",0,0.000000
"(10.0, 20.0]",14,3.856749
"(20.0, 30.0]",45,12.396694
"(30.0, 35.0]",68,18.732782
"(35.0, 40.0]",236,65.013774


In [35]:
# Antenna Height Distributions
bins = [0, 25, 50, 75, 100, 125, 150, 175, 1000]
AntennaHts = pd.DataFrame(unwired['Antenna_Ht'].value_counts(bins=bins, sort=False))
AntennaHts['Percentage (%)'] = (AntennaHts['Antenna_Ht'] /  363) * 100
AntennaHts.rename(columns={'Antenna_Ht' : 'Deployment Height (M)'}, inplace=True)
AntennaHts.to_csv('Data/AntennaHts.csv', index=True)
AntennaHts


,Deployment Height (M),Percentage (%)
"(-0.001, 25.0]",76,20.936639
"(25.0, 50.0]",209,57.575758
"(50.0, 75.0]",22,6.060606
"(75.0, 100.0]",26,7.162534
"(100.0, 125.0]",11,3.030303
"(125.0, 150.0]",6,1.652893
"(150.0, 175.0]",13,3.581267
"(175.0, 1000.0]",0,0.000000
